In [77]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min
from scipy.optimize import brentq

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div, call_delta_with_spot_and_div, call_delta_with_spot, put_delta_with_spot, put_delta_with_spot_and_div 

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [36]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
prod_type = "CE"
# prod_type = "PE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
# periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}
periods_of_interest = {2020: [4, 5]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


# Option Greeks
Normal CDF function is $\Phi(.)$ and pdf function is $\phi(.)$ <br>
$r_f$ is risk free interest rate, $q$is dividend and $r=r_f-q$ <br>
 
Call Option Delta ($\frac{dV}{dS}$): $exp(-q.dt) * \Phi(d_1)$ <br>
Put Option Delta ($\frac{dV}{dS}$): $- exp(-q.dt) * \Phi(d_1) $ <br>
Vega (Call and Put) ($\frac{dV}{d\sigma}$): $S_t * exp(-q.dt) * \phi(d_1) * \sqrt{dt} = K * exp(-r_f.dt) * \phi(d_2) * \sqrt{dt}$ <br>
Theta (Call) ($\frac{dV}{dT}$): $- exp^{(-q.dt)}\frac{S_t \phi(d_1) \sigma}{2\sqrt{dt}} - r K exp(-r.dt) \Phi(d_2) + q S exp(-q.dt) \Phi(d_1)$ <br>
Theta (put) ($\frac{dV}{dT}$): $- exp(-q.dt)\frac{S_t \phi(d_1) \sigma}{2\sqrt{dt}} + r K exp(-r.dt) \Phi(-d_2) - q S exp(-q.dt) \Phi(-d_1)$ <br>
rho (call) ($\frac{dV}{dr}$): $Kdtexp(-rdt)\Phi(d_2)$ <br>
rho (put) ($\frac{dV}{dr}$): $-Kdtexp(-rdt)\Phi(-d_2)$ <br>
gamma (call and put) ($\frac{d^{2}V}{dS^{2}}$): $ exp^{(-qdt)} \frac{\phi(d_1)}{S \sigma \sqrt{dt}} = K exp^{(-rdt)} \frac{\phi(d_2)}{S^{2}\sigma\sqrt{dt}} = \frac{Vega}{S^{2} \sigma dt}$ <br>
Vanna or DVegaDspot or DdeltaDvol (call and put) ($\frac{d^{2}V}{dSd\sigma}$): $exp^{(-qdt)} \phi(d_1) \frac{d_2}{\sigma} = \frac{Vega}{S} * (1 -\frac{d_1}{\sigma \sqrt{dt}})$ <br>
volga or vomma  (call and put) ($\frac{d^{2}V}{d\sigma^{2}}$): $Sexp^{(-qdt)}\phi(d_1)\sqrt{dt} \frac{d_1 d_2}{\sigma} = Vega * \frac{d_1 d_2}{\sigma}$

In [101]:
#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

def error_func(vol, mkt_price, stock_vec, opt_strike, r, r_f, dt):
    error = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt) - mkt_price
    return error

def error_func_short(iv, mkt_price, s_t, k, r, r_f, dt, option_type):
    if (option_type == "CE"):
        model_price = BSM_call_vec_with_div(s_t, k, r, r_f, iv, dt)
    else:
        model_price = BSM_put_vec_with_div(s_t, k, r, r_f, iv, dt)
    error = model_price - mkt_price
    return error

def find_vega(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    return (norm.pdf(d_2) * k * np.exp(-r_f * dt)) * np.sqrt(dt)

def find_gamma(s_t, k, r, r_f, vol, dt):
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    gamma = vega / ((s_t**2) * vol * dt)
    return gamma

def find_vanna(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    vanna = (vega / s_t) * (1 - ((d_1)/ (vol * np.sqrt(dt))))
    return vanna

def find_volga(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    volga = vega * (d_1 * d_2) / vol
    return volga


def find_theta_Call(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    term1 = - (np.exp(-q*dt) * s_t * norm.pdf(d_1) * vol) / (2 * np.sqrt(dt))
    term2 = - r_f * k * np.exp(-r_f*dt) * norm.cdf(d_2)
    term3 = q * s_t * np.exp(-q*dt) * norm.cdf(d_1)
    theta = term1 + term2 + term3
    return theta

# def find_theta_Call(s_t, k, r, r_f, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     q = r_f - r
#     vega = find_vega(s_t, k, r, r_f, vol, dt)
#     volga = vega * (d_1 * d_2) / vol
#     return volga


In [132]:
#7th April - ATM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 8600
mkt_price = np.array([230.25, 590.25])



iv_list = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list.append(iv)
iv_list = np.array(iv_list)


iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)


delta_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    delta_list.append(delta)
delta_list = np.array(delta_list)    


delta_cir_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[0], r_f[0], iv_list[i], dt[i])
    delta_cir_list.append(delta)
delta_cir_list = np.array(delta_cir_list)    

vega_list = []
for i in range(0,2):
    vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vega_list.append(vega)
vega_list = np.array(vega_list)

vanna_list = []
for i in range(0,2):
    vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vanna_list.append(vanna)
vanna_list = np.array(vanna_list)

volga_list = []
for i in range(0,2):
    volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    volga_list.append(volga)
volga_list = np.array(volga_list)

gamma_list = []
for i in range(0,2):
    gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    gamma_list.append(gamma)
gamma_list = np.array(gamma_list)

theta_list = []
for i in range(0,2):
    theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    theta_list.append(theta)
theta_list = np.array(theta_list)

# print(delta_list)
# print(vega_list)
# print(iv_list)
# print(iv_list_const_r)
# print(delta_list)

delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
vega_PnL = vega_list[0] * (iv_list[1] - iv_list[0])
vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) * 0.5
vanna_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list[1] - iv_list[0]))
vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
volga_PnL = volga_list[0] * ((iv_list[1] - iv_list[0])**2)
volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
theta_PnL = theta_list[0] * (dt[1] - dt[0])

realised_PnL = mkt_price[1] - mkt_price[0]

# print(vega_list)
# print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

target_opt_summary = {"Realised PnL": realised_PnL, "Delta PnL": delta_PnL, \
                     "Delta Hedge PnL": delta_hedge_PnL, "Vega PnL": vega_PnL, \
                     "Vega ConstIR PnL": vega_constIR_PnL, "gamma PnL": gamma_PnL, \
                     "Vanna PnL": vanna_PnL, "Vanna CIR PnL": vanna_cir_PnL, \
                     "Volga PnL": volga_PnL, "Volga CIR PnL": volga_cir_PnL, \
                     "Theta PnL": theta_PnL}
print(target_opt_summary)

# # Delta hedge
# stock_pos = delta*stock_vec
# deposit = mkt_price - delta*stock_vec




{'Realised PnL': 360.0, 'Delta PnL': 243.68787784390463, 'Delta Hedge PnL': 243.381642387395, 'Vega PnL': 12.047455441273742, 'Vega ConstIR PnL': 59.66323169317508, 'gamma PnL': 85.83877865519851, 'Vanna PnL': 4.225836647440411, 'Vanna CIR PnL': 20.927827641509896, 'Volga PnL': 0.07803172986640805, 'Volga CIR PnL': 1.9137893098552945, 'Theta PnL': 29.158293816600853}


In [134]:
print(realised_PnL)
theoretical_PnL = delta_PnL + vega_constIR_PnL + gamma_PnL
print(theoretical_PnL)
print(delta_cir_list)
print(gamma_list)
print((delta_cir_list[1] - delta_cir_list[0]) / (stock_vec[1] - stock_vec[0]))




360.0
389.1898881922782
[0.34399757 0.59287421]
[0.0003421  0.00034432]
0.0003513221966086555


In [110]:
print(gamma_list)

[0.0003421  0.00034432]


In [136]:
# ATM
df_hedge = pd.read_csv(input_data_path + "hedgePfl_7Apr.csv")

option_list = ["CE", "PE"]
date_list = np.unique(np.array(df_hedge["Date"]))


iv_ce_short_list = []
iv_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        mkt_price =  np.array(df["MV"])
        opt_strike = np.array(df["Strike"])
        iv_k_list = []
        for k in range(0, len_array):
#             print(option_type, opt_strike[k], mkt_price[k])
            iv = brentq(error_func_short, 0.0001, 10, args=(mkt_price[k], stock_vec[i], opt_strike[k], r[i], r_f[i], dt_short[i], option_type), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
            iv_k_list.append(iv)
        if (j == 0):
            iv_ce_short_list.append(iv_k_list)
        else:
            iv_pe_short_list.append(iv_k_list)

iv_ce_short_cir_list = []
iv_pe_short_cir_list = []

for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        mkt_price =  np.array(df["MV"])
        opt_strike = np.array(df["Strike"])
        iv_k_list = []
        for k in range(0, len_array):
            
#             print(option_type, opt_strike[k], mkt_price[k])
            iv = brentq(error_func_short, 0.0001, 10, args=(mkt_price[k], stock_vec[i], opt_strike[k], r[0], r_f[0], dt_short[i], option_type), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
#             iv_k_list.append(iv)
#             if (option_type == "CE"):
#                 model_price = BSM_call_vec_with_div(stock_vec[i], opt_strike[k], r[0], r_f[0], iv, dt_short[i])
#             else:
#                 model_price = BSM_put_vec_with_div(stock_vec[i], opt_strike[k], r[0], r_f[0], iv, dt_short[i])
#             print(model_price, mkt_price[k])
        if (j == 0):
            iv_ce_short_cir_list.append(iv_k_list)
        else:
            iv_pe_short_cir_list.append(iv_k_list)
    
            
          

delta_ce_short_list = []
delta_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        delta_k_list = []
        for k in range(0, len_array):
            if (j == 0):
                delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i])
            else:
                delta = put_delta_with_spot_and_div(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_pe_short_cir_list[i][k], dt_short[i])
            
            delta_k_list.append(delta)
    
        if (j == 0):
            delta_ce_short_list.append(delta_k_list)
        else:
            delta_pe_short_list.append(delta_k_list)
            

vega_ce_short_list = []
vega_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        vega_k_list = []
        for k in range(0, len_array):
            vega = find_vega(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i]) 
            vega_k_list.append(vega)
        if (j == 0):
            vega_ce_short_list.append(vega_k_list)
        else:
            vega_pe_short_list.append(vega_k_list)
                       

gamma_ce_short_list = []
gamma_pe_short_list = []
for i in range(0,2):
    curr_date = date_list[i]
    for j in range(0,2):
        option_type = option_list[j]  
        df = df_hedge[df_hedge["Date"] == curr_date]
        df =  df[df["Option"] == option_type]
        len_array = len(list(df["Date"]))
        opt_strike = np.array(df["Strike"])
        gamma_k_list = []
        for k in range(0, len_array):
            gamma = find_gamma(stock_vec[i], opt_strike[k], r[i], r_f[i], iv_ce_short_list[i][k], dt_short[i]) 
            gamma_k_list.append(gamma)
        if (j == 0):
            gamma_ce_short_list.append(gamma_k_list)
        else:
            gamma_pe_short_list.append(gamma_k_list)                

            
df = df_hedge[df_hedge["Date"] == date_list[0]]
df1 = df_hedge[df_hedge["Date"] == date_list[1]]
ce_weights = np.array(df[df["Option"] == "CE"]["Weights"])
pe_weights = np.array(df[df["Option"] == "PE"]["Weights"])
hedge_pfl_delta_pnl = (np.dot(np.array(delta_ce_short_list[0]), ce_weights) + np.dot(np.array(delta_pe_short_list[0]), pe_weights)) * (stock_vec[1] - stock_vec[0])
# print(hedge_pfl_delta_pnl)

ce_iv_change = np.array(iv_ce_short_list[1]) - np.array(iv_ce_short_list[0])
ce_iv_change_cir = np.array(iv_ce_short_cir_list[1]) - np.array(iv_ce_short_cir_list[0])
pe_iv_change = np.array(iv_pe_short_list[1]) - np.array(iv_pe_short_list[0])
pe_iv_change_cir = np.array(iv_pe_short_cir_list[1]) - np.array(iv_pe_short_cir_list[0])
hedge_pfl_vega_pnl = np.sum(np.multiply(np.multiply(np.array(vega_ce_short_list[0]), ce_weights), ce_iv_change)) \
                    + np.sum(np.multiply(np.multiply(np.array(vega_pe_short_list[0]), pe_weights), pe_iv_change))
hedge_pfl_vega_cir_pnl = np.sum(np.multiply(np.multiply(np.array(vega_ce_short_list[0]), ce_weights), ce_iv_change_cir)) + \
                     np.sum(np.multiply(np.multiply(np.array(vega_pe_short_list[0]), pe_weights), pe_iv_change_cir))
hedge_pfl_gamma_pnl = 0.5 * (np.dot(np.array(gamma_ce_short_list[0]), ce_weights) + np.dot(np.array(gamma_pe_short_list[0]), pe_weights)) * ((stock_vec[1] - stock_vec[0])**2)

ce_mkt_list = [np.array(df[df["Option"] == "CE"]["MV"]), np.array(df1[df1["Option"] == "CE"]["MV"])]
pe_mkt_list = [np.array(df[df["Option"] == "PE"]["MV"]), np.array(df1[df1["Option"] == "PE"]["MV"])]
hdg_pfl_realised_PnL = (np.dot(ce_mkt_list[1], ce_weights)  + np.dot(pe_mkt_list[1], pe_weights)) - \
                       (np.dot(ce_mkt_list[0], ce_weights)  + np.dot(pe_mkt_list[0], pe_weights))


print(hdg_pfl_realised_PnL, hedge_pfl_delta_pnl, hedge_pfl_vega_pnl, hedge_pfl_vega_cir_pnl, hedge_pfl_gamma_pnl)




128.74999999999955 128.75
91.74999999999909 91.75
63.5 63.5
43.850000000000136 43.85
29.100000000001273 29.1
18.29999999999518 18.3
11.400000000000318 11.4
6.7000000000000455 6.7
4.950000000000188 4.95
3.0499999999998977 3.05
2.199999999999932 2.2
1.6000000000000085 1.6
1.3000000000001108 1.3
1.1999999999985391 1.2
1.1499999999966946 1.15
0.9999999999999574 1.0
0.799999999999983 0.8
0.4500000000000206 0.45
6.549999999999869 6.55
9.25 9.25
12.74999999999983 12.75
16.59999999999974 16.6
22.55000000000439 22.55
30.900000000018622 30.9
41.85000000000127 41.85
55.40000000000032 55.4
72.7999999999995 72.8
92.90000000000009 92.9
117.99999999999909 118.0
147.29999999999927 147.3
180.75 180.75
225.30000000000064 225.3
278.1000000000431 278.1
303.54999999999836 303.55
339.0 339.0
671.6000000000076 671.6
580.5499999999993 580.55
482.9499999999962 482.95
396.04999999999836 396.05
314.6000000000686 314.6
239.55000000001473 239.55
176.64999999999873 176.65
124.65000000000055 124.65
84.64999999999964

In [131]:
print(hdg_pfl_realised_PnL)
hdg_pfl_theoretical_PnL = hedge_pfl_delta_pnl + hedge_pfl_vega_cir_pnl + hedge_pfl_gamma_pnl
print(hdg_pfl_theoretical_PnL)

359.45918622280004
458.6402069468196


In [ ]:
def put_delta_with_spot(s_t, k, r, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    dc_ds = norm.cdf(d_1)
    # put delta = call delta - 1
    return (dc_ds-1)

def put_delta_with_spot_and_div(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    q = r_f - r 
    dc_ds = np.exp(- q * dt) * norm.cdf(d_1)
    return (dc_ds-np.exp(-q*dt))

# Function to obtain gama with respect to spot
def gamma_with_spot(s_t, k, r, vol, dt):
    d_1_g = (1 / (vol * (dt ** 0.5)))
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_g * d1_log
    gamma = norm.pdf(d_1) * (1 / (vol * s_t * dt)) 
    return gamma


{'30-Apr-2020': [datetime.date(2020, 3, 26), datetime.date(2020, 4, 1), datetime.date(2020, 4, 9), datetime.date(2020, 4, 16), datetime.date(2020, 4, 23)], '28-May-2020': [datetime.date(2020, 4, 30), datetime.date(2020, 5, 7), datetime.date(2020, 5, 14), datetime.date(2020, 5, 21)]}


In [118]:
#7th April - ITM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 8000
mkt_price = np.array([526.15, 1045.55])



iv_list = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list.append(iv)
iv_list = np.array(iv_list)


iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)


delta_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    delta_list.append(delta)
delta_list = np.array(delta_list)    


vega_list = []
for i in range(0,2):
    vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vega_list.append(vega)
vega_list = np.array(vega_list)

vanna_list = []
for i in range(0,2):
    vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vanna_list.append(vanna)
vanna_list = np.array(vanna_list)

volga_list = []
for i in range(0,2):
    volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    volga_list.append(volga)
volga_list = np.array(volga_list)

gamma_list = []
for i in range(0,2):
    gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    gamma_list.append(gamma)
gamma_list = np.array(gamma_list)

theta_list = []
for i in range(0,2):
    theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    theta_list.append(theta)
theta_list = np.array(theta_list)

# print(delta_list)
# print(vega_list)
# print(iv_list)
# print(iv_list_const_r)
# print(delta_list)

delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
vega_PnL = vega_list[0] * (iv_list[1] - iv_list[0])
vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) / 2
vanna_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list[1] - iv_list[0]))
vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
volga_PnL = volga_list[0] * ((iv_list[1] - iv_list[0])**2)
volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
theta_PnL = theta_list[0] * (dt[1] - dt[0])

actual_PnL = mkt_price[1] - mkt_price[0]

# print(vega_list)
# print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

target_opt_summary = {"Realised PnL": actual_PnL, "Delta PnL": delta_PnL, \
                     "Delta Hedge PnL": delta_hedge_PnL, "Vega PnL": vega_PnL, \
                     "Vega ConstIR PnL": vega_constIR_PnL, "gamma PnL": gamma_PnL, \
                     "Vanna PnL": vanna_PnL, "Vanna CIR PnL": vanna_cir_PnL, \
                     "Volga PnL": volga_PnL, "Volga CIR PnL": volga_cir_PnL, \
                     "Theta PnL": theta_PnL}
print(target_opt_summary)

# # Delta hedge
# stock_pos = delta*stock_vec
# deposit = mkt_price - delta*stock_vec




{'Realised PnL': 519.4, 'Delta PnL': 393.92662548235694, 'Delta Hedge PnL': 393.44846108652837, 'Vega PnL': 22.910122308486457, 'Vega ConstIR PnL': 102.44770655578284, 'gamma PnL': 80.86706709687205, 'Vanna PnL': 0.07974845652495668, 'Vanna CIR PnL': 0.3566129574663592, 'Volga PnL': -0.0009506221852992682, 'Volga CIR PnL': -0.019008927359186384, 'Theta PnL': 35.43972541322724}


In [119]:
393 + 102

495

In [120]:
#7th April - OTM
stock_vec = np.array([8083.8, 8792.2])
futures_price = np.array([8084.5, 8875.85])
dt = np.array([((date(2020,4,30) - date(2020, 4, 3)).days)/365, ((date(2020,4,30) - date(2020, 4, 7)).days)/365])
dt_short = np.array([((date(2020,4,9) - date(2020, 4, 3)).days)/365, ((date(2020,4,9) - date(2020, 4, 7)).days)/365])
r_f = np.array([0.04, 0.04])
r = np.log(np.divide(futures_price, stock_vec))/dt
q = r_f - r
opt_strike = 9500
mkt_price = np.array([44.4, 150.55])



iv_list = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[i], r_f[i], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list.append(iv)
iv_list = np.array(iv_list)


iv_list_const_r = []
for i in range(0,2):
    iv = brentq(error_func, 0.0001, 1, args=(mkt_price[i], stock_vec[i], opt_strike, r[0], r_f[0], dt[i]), xtol=2e-12, rtol=8.881784197001252e-16, maxiter=100, full_output=False, disp=True)
    iv_list_const_r.append(iv)
iv_list_const_r = np.array(iv_list_const_r)


delta_list = []
for i in range(0,2):
    delta = call_delta_with_spot_and_div(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    delta_list.append(delta)
delta_list = np.array(delta_list)    


vega_list = []
for i in range(0,2):
    vega = find_vega(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vega_list.append(vega)
vega_list = np.array(vega_list)

vanna_list = []
for i in range(0,2):
    vanna = find_vanna(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    vanna_list.append(vanna)
vanna_list = np.array(vanna_list)

volga_list = []
for i in range(0,2):
    volga = find_volga(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    volga_list.append(volga)
volga_list = np.array(volga_list)

gamma_list = []
for i in range(0,2):
    gamma = find_gamma(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    gamma_list.append(gamma)
gamma_list = np.array(gamma_list)

theta_list = []
for i in range(0,2):
    theta = find_theta_Call(stock_vec[i], opt_strike, r[i], r_f[i], iv_list[i], dt[i])
    theta_list.append(theta)
theta_list = np.array(theta_list)

# print(delta_list)
# print(vega_list)
# print(iv_list)
# print(iv_list_const_r)
# print(delta_list)

delta_hedge_pos = np.array([delta_list[0] * stock_vec[0], mkt_price[0] - delta_list[0]*stock_vec[1]])
delta_PnL = delta_list[0] * (stock_vec[1] - stock_vec[0])
delta_hedge_PnL = ((delta_hedge_pos[0] * stock_vec[1]) / stock_vec[0]) + delta_hedge_pos[1] * np.exp(r_f[0]/365) - np.sum(delta_hedge_pos)
vega_PnL = vega_list[0] * (iv_list[1] - iv_list[0])
vega_constIR_PnL = vega_list[0] * (iv_list_const_r[1] - iv_list_const_r[0])
gamma_PnL = gamma_list[0] * ((stock_vec[1] - stock_vec[0])**2) / 2
vanna_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list[1] - iv_list[0]))
vanna_cir_PnL = vanna_list[0] * ((stock_vec[1] - stock_vec[0]) * (iv_list_const_r[1] - iv_list_const_r[0]))
volga_PnL = volga_list[0] * ((iv_list[1] - iv_list[0])**2)
volga_cir_PnL = volga_list[0] * ((iv_list_const_r[1] - iv_list_const_r[0])**2)
theta_PnL = theta_list[0] * (dt[1] - dt[0])

actual_PnL = mkt_price[1] - mkt_price[0]

# print(vega_list)
# print(actual_PnL, delta_PnL, delta_hedge_PnL, vega_PnL, vega_constIR_PnL)

target_opt_summary = {"Realised PnL": actual_PnL, "Delta PnL": delta_PnL, \
                     "Delta Hedge PnL": delta_hedge_PnL, "Vega PnL": vega_PnL, \
                     "Vega ConstIR PnL": vega_constIR_PnL, "gamma PnL": gamma_PnL, \
                     "Vanna PnL": vanna_PnL, "Vanna CIR PnL": vanna_cir_PnL, \
                     "Volga PnL": volga_PnL, "Volga CIR PnL": volga_cir_PnL, \
                     "Theta PnL": theta_PnL}
print(target_opt_summary)

# # Delta hedge
# stock_pos = delta*stock_vec
# deposit = mkt_price - delta*stock_vec




{'Realised PnL': 106.15, 'Delta PnL': 71.35090516012212, 'Delta Hedge PnL': 71.25871807900296, 'Vega PnL': -9.669603662056714, 'Vega ConstIR PnL': 2.1853671507241925, 'gamma PnL': 45.314885657870185, 'Vanna PnL': -9.800502067700691, 'Vanna CIR PnL': 2.2149506875240528, 'Volga PnL': 0.9681721049361801, 'Volga CIR PnL': 0.04945202040984683, 'Theta PnL': 12.737096391642162}
